In [ ]:
pip install transformers torch pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

 distilbert-base-uncased because it’s a lightweight, faster version of BERT that still gives good accuracy for NLP tasks like intent classification, entity recognition, or sentiment analysis in your fast food chatbot.

Reasons why your sir chose it:

Smaller & Faster → DistilBERT has ~40% fewer parameters than BERT but runs much faster, which is good for a chatbot that needs quick replies.

Good Accuracy → Despite being smaller, it retains ~95–97% of BERT’s performance.

Uncased → The model ignores letter casing (“Pizza” = “pizza”), which is useful in casual chat where users may type in different cases.

Pretrained → Already trained on large text corpora, so you don’t need huge datasets to fine-tune it.

Suitable for Deployment → Lightweight models are easier to deploy on limited resources (laptops, servers, even cloud).

In [3]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

# Load combined dataset
df = pd.read_csv("chatbot_combined_dataset.csv")
queries = df["Query"].values
intents = df["Intent"].values

# Encode intents
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(intents)

# Split into train and validation sets (80-20 split)
train_texts, val_texts, train_labels, val_labels = train_test_split(queries, labels, test_size=0.2, random_state=42)

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_encoder.classes_))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Metric computation function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [7]:
# Tokenize data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Create dataset class
class ChatbotDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ChatbotDataset(train_encodings, train_labels)
val_dataset = ChatbotDataset(val_encodings, val_labels)

In [8]:
# Training arguments with optimized saving
training_args = TrainingArguments(
    output_dir="./chatbot_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,  # Only keeps the best model
    fp16=True,  # Uses mixed precision training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train and save optimally
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.449076,0.583333,0.543339,0.549399,0.583333
2,No log,0.963348,0.854167,0.859442,0.891369,0.854167
3,No log,0.813146,0.875000,0.876224,0.901786,0.875000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=72, training_loss=1.267158932156033, metrics={'train_runtime': 24.9276, 'train_samples_per_second': 23.107, 'train_steps_per_second': 2.888, 'total_flos': 2831793619968.0, 'train_loss': 1.267158932156033, 'epoch': 3.0})

In [9]:
# Load test dataset
test_df = pd.read_csv("Test_Data_Sample.csv")
test_queries = test_df["Query"].values
test_labels = label_encoder.transform(test_df["Intent"].values)

# Tokenize test data
test_encodings = tokenizer(list(test_queries), truncation=True, padding=True, max_length=128)
test_dataset = ChatbotDataset(test_encodings, test_labels)

# Evaluate
eval_results = trainer.evaluate(test_dataset)
print("Test Evaluation Results:", eval_results)

Test Evaluation Results: {'eval_loss': 0.6991308331489563, 'eval_accuracy': 0.96, 'eval_f1': 0.9597202797202797, 'eval_precision': 0.9657142857142857, 'eval_recall': 0.96, 'eval_runtime': 0.6435, 'eval_samples_per_second': 77.695, 'eval_steps_per_second': 10.877, 'epoch': 3.0}


In [13]:
# Explicitly save only what's needed
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

print("Model trained and efficiently saved!")

Model trained and efficiently saved!


In [15]:
from google.colab import files
!zip -r final_model.zip final_model
files.download("final_model.zip")

updating: final_model/ (stored 0%)
updating: final_model/config.json (deflated 54%)
updating: final_model/vocab.txt (deflated 53%)
updating: final_model/tokenizer_config.json (deflated 75%)
updating: final_model/model.safetensors (deflated 8%)
updating: final_model/special_tokens_map.json (deflated 42%)
updating: final_model/training_args.bin (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import pandas as pd
import sqlite3
import random
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
import gradio as gr


In [20]:
import pandas as pd
import sqlite3
import random
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
import gradio as gr

# Load fine-tuned model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("./final_model")
model = DistilBertForSequenceClassification.from_pretrained("./final_model")

# Load label encoder
df = pd.read_csv("chatbot_combined_dataset.csv")
label_encoder = LabelEncoder()
label_encoder.fit(df["Intent"].values)

# Initialize SQLite database
def init_db():
    with sqlite3.connect("fast_food.db") as conn:
        cursor = conn.cursor()
        # Create menu table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS menu (
                item_id INTEGER PRIMARY KEY,
                item_name TEXT,
                price REAL,
                category TEXT
            )
        ''')
        # Insert menu items
        menu_items = [
            ("Veg Rice", 50, "Vegetarian"),
            ("Veg Noodles", 50, "Vegetarian"),
            ("Veg Manchurian", 50, "Vegetarian"),
            ("Chicken Rice", 80, "Non-Vegetarian"),
            ("Chicken Noodles", 80, "Non-Vegetarian"),
            ("Chicken Manchurian", 80, "Non-Vegetarian")
        ]
        cursor.executemany("INSERT OR IGNORE INTO menu (item_name, price, category) VALUES (?, ?, ?)", menu_items)
        # Create orders table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS orders (
                order_id INTEGER,
                item_name TEXT,
                quantity INTEGER,
                total_price REAL
            )
        ''')
        conn.commit()

# Calculate bill
def calculate_bill(order_id):
    with sqlite3.connect("fast_food.db") as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT SUM(total_price) FROM orders WHERE order_id = ?", (order_id,))
        total = cursor.fetchone()[0]
        return total if total else 0

# Chatbot function
def chatbot_response(user_input, order_id, chat_history):
    # Tokenize input
    encodings = tokenizer(user_input, truncation=True, padding=True, max_length=128, return_tensors="pt")
    outputs = model(**encodings)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    intent = label_encoder.inverse_transform([predicted_label])[0]

    # Initialize order_id if None
    if order_id is None and intent in ["add item", "remove item", "confirm order", "check order status"]:
        order_id = random.randint(1000, 9999)

    # Handle intents
    if intent == "greet":
        return "", order_id, chat_history + [(user_input, "Hello! I’m Fast Food Chatbot, how can I assist you?")]
    elif intent == "menu":
        with sqlite3.connect("fast_food.db") as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT item_name, price, category FROM menu")
            items = cursor.fetchall()
            veg = [f"{item[0]} (${item[1]})" for item in items if item[2] == "Vegetarian"]
            non_veg = [f"{item[0]} (${item[1]})" for item in items if item[2] == "Non-Vegetarian"]
            response = f"Our menu: Vegetarian - {', '.join(veg)}; Non-Vegetarian - {', '.join(non_veg)}."
        return "", order_id, chat_history + [(user_input, response)]
    elif intent == "add item":
        items = []
        if "2 veg noodles" in user_input.lower():
            items.append(("Veg Noodles", 2, 50 * 2))
        elif "1 chicken rice" in user_input.lower():
            items.append(("Chicken Rice", 1, 80 * 1))
        elif "1 veg rice" in user_input.lower() and "1 chicken noodles" in user_input.lower():
            items.append(("Veg Rice", 1, 50 * 1))
            items.append(("Chicken Noodles", 1, 80 * 1))
        # Add more conditions for other combinations as needed
        if items:
            with sqlite3.connect("fast_food.db") as conn:
                cursor = conn.cursor()
                for item_name, quantity, total_price in items:
                    cursor.execute("INSERT INTO orders (order_id, item_name, quantity, total_price) VALUES (?, ?, ?, ?)",
                                  (order_id, item_name, quantity, total_price))
                conn.commit()
            response = f"Your order of {', '.join([f'{q} {n}' for n, q, _ in items])} is placed with bill ${calculate_bill(order_id)} and your ID is {order_id}."
            return "", order_id, chat_history + [(user_input, response)]
        return "", order_id, chat_history + [(user_input, "Please specify items to add (e.g., 2 veg noodles).")]
    elif intent == "remove item":
        if order_id:
            with sqlite3.connect("fast_food.db") as conn:
                cursor = conn.cursor()
                cursor.execute("SELECT item_name, quantity FROM orders WHERE order_id = ?", (order_id,))
                items = cursor.fetchall()
                if "veg noodles" in user_input.lower():
                    cursor.execute("DELETE FROM orders WHERE order_id = ? AND item_name = ?", (order_id, "Veg Noodles"))
                elif "chicken rice" in user_input.lower():
                    cursor.execute("DELETE FROM orders WHERE order_id = ? AND item_name = ?", (order_id, "Chicken Rice"))
                # Add more conditions as needed
                conn.commit()
                response = f"Items removed from order ID {order_id}. New bill: ${calculate_bill(order_id)}."
                return "", order_id, chat_history + [(user_input, response)]
        return "", order_id, chat_history + [(user_input, "Please provide an order ID to remove items.")]
    elif intent == "confirm order":
        if order_id:
            response = f"Order ID {order_id} confirmed. Final bill: ${calculate_bill(order_id)}."
            return "", order_id, chat_history + [(user_input, response)]
        return "", order_id, chat_history + [(user_input, "Please provide an order ID to confirm.")]
    elif intent == "check order status":
        if order_id:
            response = f"Order ID {order_id} is {'ready for pickup' if random.random() > 0.5 else 'being prepared'}."
            return "", order_id, chat_history + [(user_input, response)]
        return "", order_id, chat_history + [(user_input, "Please provide an order ID to check status.")]
    elif intent == "end greeting":
        return "", None, chat_history + [(user_input, "Goodbye! Come back soon for more tasty food!")]
    elif intent == "unrelated":
        return "", order_id, chat_history + [(user_input, "Sorry, I don’t understand you. I can assist with food orders and billing.")]
    return "", order_id, chat_history + [(user_input, "I’m not sure what you mean. Try asking about the menu or placing an order!")]

# Initialize database
init_db()

# Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("# Fast Food Chatbot")
    order_id = gr.State(value=None)
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type your message (e.g., 'Hi', 'Show menu', 'Add 2 veg noodles')")
    clear = gr.Button("Clear")

    def respond(message, chat_history, order_id):
        new_message, new_order_id, new_chat_history = chatbot_response(message, order_id, chat_history)
        return new_message, new_chat_history, new_order_id

    msg.submit(respond, [msg, chatbot, order_id], [msg, chatbot, order_id])
    clear.click(lambda: (None, None), None, [chatbot, order_id], queue=False)

interface.launch()
print("Chatbot launched with Gradio and thread-safe SQLite integration!")

/tmp/ipython-input-1241448094.py:142: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a4af291d4cb55a781.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Chatbot launched with Gradio and thread-safe SQLite integration!
